
## Ingest Resuts.json file to parquet
Path : dbfs:/mnt/formula1dlsaga/raw/results.json

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType, TimestampType, FloatType
from pyspark.sql.functions import lit, col, current_timestamp

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date= dbutils.widgets.get("p_file_date")


In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("grid", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("positionText", StringType(), True),
                                      StructField("positionOrder", IntegerType(), True),
                                      StructField("points", FloatType(), True),
                                      StructField("laps", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True),
                                      StructField("fastestLap", IntegerType(), True),
                                      StructField("rank", IntegerType(), True),
                                      StructField("fastestLapTime", StringType(), True),
                                      StructField("fastestLapSpeed", FloatType(), True),
                                      StructField("statusId", StringType(), True)])

In [0]:
results_df = spark.read.json(f"{raw_folder_path}/{v_file_date}/results.json", schema=results_schema)

In [0]:
results_df_func = results_df.withColumnRenamed("resultId", "result_id") \
                                      .withColumnRenamed("raceId", "race_id") \
                                      .withColumnRenamed("driverId", "driver_id") \
                                      .withColumnRenamed("constructorId", "constructor_id") \
                                      .withColumnRenamed("positionText", "position_text") \
                                      .withColumnRenamed("positionOrder", "position_order") \
                                      .withColumnRenamed("fastestLap", "fastest_lap") \
                                      .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                      .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                      .withColumn("data_source", lit(v_data_source))\
                                      .withColumn("file_date", lit(v_file_date)) 

In [0]:
results_df_final = add_ingestion_date(results_df_func)


# Method 1

In [0]:
# for race_id_list in results_df_final.select("race_id").distinct().collect():
#     if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#         spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")

In [0]:
# results_df_final.write.parquet(f"{processed_folder_path}/results", 
#                                         mode="overwrite",
#                                         partitionBy="race_id")

In [0]:
# results_df_final.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")

# Method 2

In [0]:
%sql
--DROP TABLE f1_processed.results;

In [0]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [0]:
# results_df_final = results_df_final.select("result_id",
#                                            "driver_id",
#                                            "constructor_id",
#                                            "number",
#                                            "grid",
#                                            "position",
#                                            "position_text",
#                                            "position_order",
#                                            "points",
#                                            "laps",
#                                            "time",
#                                            "milliseconds",
#                                            "fastest_lap",
#                                            "rank",
#                                            "fastest_lap_time",
#                                            "fastest_lap_speed",
#                                            "data_source",
#                                            "file_date",
#                                            "ingestion_date",
#                                            "race_id")

In [0]:
results_df_final = move_column_to_last(results_df_final, "race_id")


Dedupe the df

In [0]:
results_deduped_df = results_df_final.dropDuplicates(["race_id", "driver_id"])

In [0]:
# if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#     results_df_final.write.mode("overwrite").insertInto("f1_processed.results")
# else:
#     results_df_final.write.mode("overwrite").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.results")

In [0]:
# write_to_database(results_df_final, "f1_processed", "results", "race_id")

In [0]:
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_data(results_deduped_df, 'f1_processed', 'results', processed_folder_path, merge_condition, 'race_id')

In [0]:
%sql
SELECT race_id, COUNT(1) FROM f1_processed.results GROUP BY race_id ORDER BY race_id DESC;

In [0]:
# df = spark.read.parquet(f"{processed_folder_path}/results")
# display(df)

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
SELECT COUNT(1) FROM f1_processed.results WHERE file_date='2021-03-21';

In [0]:
%sql
SELECT race_id, driver_id, COUNT(1) 
FROM f1_processed.results
GROUP BY race_id, driver_id
HAVING COUNT(1) > 1
ORDER BY race_id, driver_id DESC;

In [0]:
%sql
SELECT * FROM f1_processed.results WHERE race_id = 540 AND driver_id = 229;